In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/SCH_Proposal

/content/drive/My Drive/Colab Notebooks/SCH_Proposal


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# For BART, T5, Pegasus
#!pip3 install -q transformers==3.5.0
#!pip3 install -q transformers==4.15.0
!pip3 install -q transformers

!pip3 install -q torch==1.7.0
!pip3 install -q sentencepiece

     |████████████████████████████████| 3.4 MB 8.2 MB/s 
     |████████████████████████████████| 895 kB 61.1 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 3.3 MB 13.5 MB/s 
     |████████████████████████████████| 61 kB 591 kB/s 
     |████████████████████████████████| 776.7 MB 4.4 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
     |████████████████████████████████| 1.2 MB 7.9 MB/s 


In [ ]:
# Checking out the GPU we have access to
!nvidia-smi

Sat Jan  1 17:33:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install -q scispacy
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

!python3 -m spacy download en

     |████████████████████████████████| 44 kB 2.3 MB/s 
     |████████████████████████████████| 13.5 MB 6.4 MB/s 
     |████████████████████████████████| 6.3 MB 26.7 MB/s 
     |████████████████████████████████| 71 kB 9.8 MB/s 
     |████████████████████████████████| 188 kB 63.4 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 10.1 MB 51.1 MB/s 
     |████████████████████████████████| 628 kB 62.2 MB/s 
     |████████████████████████████████| 451 kB 62.1 MB/s 
     |████████████████████████████████| 33.1 MB 62.2 MB/s 
     |████████████████████████████████| 125.1 MB 38 kB/s 
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7 MB 9.1 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/SCH_Proposal


In [ ]:
!pip3 install -q jsonlines

In [ ]:
import os
import numpy as np
import pandas as pd
import json
import jsonlines
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import (T5Tokenizer, T5ForConditionalGeneration, 
                          BartTokenizer, BartForConditionalGeneration,
                          PegasusTokenizer, PegasusForConditionalGeneration,
                          ProphetNetTokenizer, ProphetNetForConditionalGeneration, ProphetNetConfig)

from transformers import AutoModel, AutoTokenizer

In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(device)

cuda


## Read input args file

In [ ]:
with open("input_args.json") as fp:
  input_args = json.load(fp)
fp.close()

In [ ]:
input_args

{'DATA_PATH': './',
 'EPOCHS': 30,
 'LEARNING_RATE': 0.0001,
 'MAX_LEN': 512,
 'SUMMARY_LEN': 210,
 'TRAIN_BATCH_SIZE': 1,
 'VALID_BATCH_SIZE': 1,
 'alpha': 0.02,
 'data_name': 'chest-x-rays',
 'filename': 'NLMCXR_reports_ecgen_radiology_w_named_entites.jsonl',
 'model': 'T5',
 'model_type': 'w_named_entities',
 'train_size': 0.8,
 'w_named_entity': True}

### Read and parse the pubmed dataset

In [ ]:
# If model training is required with two or more cost functions, this function shall return two or more dataframes for all
# input-to-output mapping

def _read_parse_data(filepath, w_named_entity=False):
  data, final_data, final_named_entity_chain = [], [], []
  with open(filepath) as f:
    for line in f:
      data.append(json.loads(line))
  f.close()

  if w_named_entity:
    for article in data:
      finding = "".join(article['finding'])
      impression = "".join(article['impression']).replace('<S>', '').replace('</S>', '')
      final_data.append([finding, impression])   # this is the source article to summary mapping dataframe

      finding_entities = " ".join(article['finding_entities'].split(' | '))
      impression_entities = " ".join(article['impression_entities'].split(' | '))
      final_named_entity_chain.append([finding_entities, impression_entities])   # for mapping of finding entities and impression entities
      

    #df = pd.DataFrame(final_data, columns=['finding', 'impression', 'finding_entities'])
    df_source_article = pd.DataFrame(final_data, columns=['finding', 'impression'])
    df_source_article.finding = 'summarize: ' + df_source_article.finding


    #finding_entities = df["finding_entities"]
    df_entity_chain = pd.DataFrame(final_named_entity_chain, columns=['finding_entities', 'impression_entities'])
    df_entity_chain.finding_entities = 'summarize: ' + df_entity_chain.finding_entities

    df_entity_chain.rename(columns={"finding_entities" : "finding", "impression_entities" : "impression"}, inplace=True)

    return df_source_article, df_entity_chain

  else:
    for article in data:
      impression = "".join(article['impression']).replace('<S>', '').replace('</S>', '')
      finding = "".join(article['finding'])
      final_data.append([finding, impression])

    df_source_article = pd.DataFrame(final_data, columns=['finding', 'impression'])
    df_source_article.finding = 'summarize: ' + df_source_article.finding

    return df_source_article

In [ ]:
def main():
  DATA_PATH = input_args['DATA_PATH']
  filename = input_args['filename']
  filepath = f"{DATA_PATH}/{filename}"

  w_named_entity = input_args['w_named_entity']

  # call to the method above
  if w_named_entity:
    df_source_article, df_entity_chain = _read_parse_data(filepath, w_named_entity)

    return df_source_article, df_entity_chain

  else:
    df_source_article = _read_parse_data(filepath, w_named_entity)

    return df_source_article


In [ ]:
if __name__ == "__main__":
  w_named_entity = input_args['w_named_entity']
  if w_named_entity:
    df_source_article, df_entity_chain = main()
  else:
    df_source_article = main()

In [ ]:
df_source_article.head()

,finding,impression
0,summarize: EXAM: XR Chest 1 View Frontal . [...,1. New diffuse bilateral alveolar opacificat...
1,summarize: EXAM: Chest one view frontal [PER...,Interval resolution of the findings of pulmona...
2,summarize: EXAM: 1. [PERSONALNAME] OF THE CH...,CHEST: 1. No evidence for metastatic dise...
3,summarize: PROCEDURE: XR CHEST [ADDRESS] [PER...,1. Limited portable chest radiograph wit...
4,summarize: PROCEDURE: [PERSONALNAME] BRAIN WI...,1. No acute intracranial abnormality. If cl...


In [ ]:
if input_args['w_named_entity']:
  print(df_entity_chain.head())

                                             finding                                         impression
0  summarize: pneumothorax increased bilateral al...  New diffuse bilateral alveolar opacifications ...
1  summarize: Residual linear density left base t...                    pulmonary edema new infiltrates
2  summarize: hypodensities noncircumferential fr...  Subtle irregular thickening metastatic disease...
3  summarize: stable linear opacities at the lung...  This study Limited portable chest radiograph a...
4  summarize: altered mental status acute large v...  infarct acute intracranial abnormality chronic...


## Select which tokenizer and model to fine-tune

In [ ]:
# Defining some key variables that will be used later on in the training

MAX_LEN = input_args['MAX_LEN']
SUMMARY_LEN = input_args['SUMMARY_LEN']
TRAIN_BATCH_SIZE = input_args['TRAIN_BATCH_SIZE']
VALID_BATCH_SIZE = input_args['VALID_BATCH_SIZE']
EPOCHS = input_args['EPOCHS']
LEARNING_RATE = input_args['LEARNING_RATE']

In [ ]:
# Sections of config
def _selectModel(modelName):
  if modelName.lower() == "t5":
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
  elif modelName.lower() == 'bart':
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
  elif modelName.lower() == 'pegasus':
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
    model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
  elif modelName.lower() == 'prophetnet':
    tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased-cnndm')
    model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased-cnndm')
  elif modelName.lower() == 'unilm':
    tokenizer = AutoTokenizer.from_pretrained("microsoft/unilm-base-cased")
    model = AutoModel.from_pretrained("microsoft/unilm-base-cased")
  elif modelName.lower() == 'longformer':
    tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
    model = LEDForConditionalGeneration.from_pretrained('allenai/led-base-16384')

  model = model.to(device)

  return model, tokenizer

In [ ]:
def main():
  modelName = input_args["model"]
  model, tokenizer = _selectModel(modelName)

  return model, tokenizer

In [ ]:
if __name__ == "__main__":
  model, tokenizer = main()

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

### Instantiate an optimizer

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage 
# for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.impression = self.data.impression
        self.finding = self.data.finding

    def __len__(self):
        return len(self.impression)

    def __getitem__(self, index):
        finding = str(self.finding[index])
        finding = ' '.join(finding.split())

        impression = str(self.impression[index])
        impression = ' '.join(impression.split())

        source = self.tokenizer.batch_encode_plus([finding], max_length= self.source_len, 
                                                  truncation=True, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([impression], max_length= self.summ_len, 
                                                  truncation=True, pad_to_max_length=True,return_tensors='pt')


        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

w_named_entity = input_args['w_named_entity']

train_size = input_args['train_size']
train_dataset=df_source_article.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=df_source_article.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df_source_article.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)


# for the finding_entities to impression_entities mapping
if w_named_entity:
  train_size = input_args['train_size']
  train_dataset_entity_chain=df_entity_chain.sample(frac=train_size,random_state=42).reset_index(drop=True)
  test_dataset_entity_chain=df_entity_chain.drop(train_dataset_entity_chain.index).reset_index(drop=True)


  print("FULL Dataset: {}".format(df_entity_chain.shape))
  print("TRAIN Dataset: {}".format(train_dataset_entity_chain.shape))
  print("TEST Dataset: {}".format(test_dataset_entity_chain.shape))

  training_set_entity_chain = CustomDataset(train_dataset_entity_chain, tokenizer, MAX_LEN, SUMMARY_LEN)
  testing_set_entity_chain = CustomDataset(test_dataset_entity_chain, tokenizer, MAX_LEN, SUMMARY_LEN)



FULL Dataset: (6182, 2)
TRAIN Dataset: (4946, 2)
TEST Dataset: (1236, 2)
FULL Dataset: (6182, 2)
TRAIN Dataset: (4946, 2)
TEST Dataset: (1236, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


# for the finding_entities to impression_entities mapping
w_named_entity = input_args['w_named_entity']

if w_named_entity:
  training_loader_entity_chain = DataLoader(training_set_entity_chain, **train_params)
  testing_loader_entity_chain = DataLoader(testing_set_entity_chain, **test_params)



In [ ]:
def train(epoch, w_named_entity=False, alpha = 0.5):
  model.train()
  
  if w_named_entity:
    for iter, (source_data, data_entity_chain) in enumerate(zip(training_loader, training_loader_entity_chain), 0):
      y = source_data['target_ids'].to(device, dtype = torch.long)
      y_ids = y[:, :-1].contiguous()
      lm_labels = y[:, 1:].clone().detach()
      lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
      ids = source_data['source_ids'].to(device, dtype = torch.long)
      mask = source_data['source_mask'].to(device, dtype = torch.long)

      outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

      total_loss = outputs[0]   # for the vanilla training configuration loss

      y = data_entity_chain['target_ids'].to(device, dtype = torch.long)
      y_ids = y[:, :-1].contiguous()
      lm_labels = y[:, 1:].clone().detach()
      lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
      ids = data_entity_chain['source_ids'].to(device, dtype = torch.long)
      mask = data_entity_chain['source_mask'].to(device, dtype = torch.long)

      outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

      total_loss = alpha * total_loss + alpha * outputs[0]   # total loss (i.e., with the one from the source article to summary mapping)

    
      if iter % 500==0:
          print(f'Epoch: {epoch}, Loss:  {total_loss.item()}')
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

  else:
    for iter, source_data in enumerate(training_loader,  0):
      y = source_data['target_ids'].to(device, dtype = torch.long)
      y_ids = y[:, :-1].contiguous()
      lm_labels = y[:, 1:].clone().detach()
      lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
      ids = source_data['source_ids'].to(device, dtype = torch.long)
      mask = source_data['source_mask'].to(device, dtype = torch.long)

      outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

      total_loss = outputs[0]

      if iter % 500==0:
          print(f'Epoch: {epoch}, Loss:  {total_loss.item()}')
      
      optimizer.zero_grad()
      total_loss.backward()
      optimizer.step()


In [ ]:
!pip3 install -q torch==1.7.0
!pip3 install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import torch
import transformers
import os

### Call to the train method above

In [ ]:
def main(model):
  w_named_entity = input_args['w_named_entity']

  for epoch in range(EPOCHS):
    train(epoch, w_named_entity)

  model_name = input_args['model']
  model_type = input_args["model_type"]   # changes with type (i.e., vanilla, w_named_entities, w_named_entities_w_facts)
  
  data_name = input_args["data_name"]

  model_path = f"{data_name}-FINAL-CHECKPTS/{model_name}"
  os.makedirs(model_path, exist_ok=True)

  # Save entire model to a path
  final_model_path = f"{model_path}/{model_name}-{model_type}.pt"
  torch.save(model, final_model_path)


  # Saving checkpoints
  # model path
  MODEL_DIRECTORY_PATH = f"{data_name}-FINAL-CHECKPTS/{model_name}"

  model_path = f"{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}.pt"
  model = torch.load(model_path)

  os.makedirs(f'{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}-checkpoints', exist_ok=True)

  model.save_pretrained(f'{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}-checkpoints/')


In [ ]:
if __name__ == "__main__":
  main(model)

### Loading entire model from the file

In [ ]:
!pip3 install -q torch==1.7.0
!pip3 install -q git+https://github.com/huggingface/transformers

In [ ]:
import torch
import transformers
import os

In [ ]:
from torchsummary import summary

model_name = "T5"
model_type = "vanilla"
data_name = "heart-failure"

# model path
MODEL_DIRECTORY_PATH = f"{data_name}-FINAL-CHECKPTS/{model_name}"
model_path = f"{MODEL_DIRECTORY_PATH}/{model_name}-{model_type}.pt"

#model = model.load_state_dict(torch.load(model_path))
model = torch.load(model_path)

### Experiment with generating summaries. Note that during inference, we shall experiment with the different input configurations

In [ ]:
import spacy
import scispacy
import pandas as pd
import numpy as np
import pickle as pk
from pprint import pprint
import os

from spacy import displacy
import en_core_sci_sm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker

In [ ]:
def _loadModel(model_name, model_path):
  if model_name == 'BART':
    model = BartForConditionalGeneration.from_pretrained(model_path)
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
  elif model_name == 'T5':
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
  elif model_name == "Pegasus":
    model = PegasusForConditionalGeneration.from_pretrained(model_path)
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

  return model, tokenizer

In [ ]:
def _generate_abstractive_summary(input_text, model, tokenizer, MAX_LEN=250):
  inputs = tokenizer(input_text, truncation=True, padding='longest',return_tensors='pt').to(device)
  # Get the highest-scoring beam as the abstractive summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=70, max_length=MAX_LEN, 
                               early_stopping=True, num_return_sequences=1)
  
  tgt_text = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
  abstractive_summary = tgt_text[0]

  return abstractive_summary


### Install stanza to run NER on the generated summaries

In [ ]:
!pip3 install -q git+https://github.com/stanfordnlp/stanza.git

     |████████████████████████████████| 170 kB 9.3 MB/s 


In [ ]:
import stanza

def _extract_named_entities(clinical_note, i2b2_model):
  lst_entities = []

  doc_i2b2 = i2b2_model(clinical_note)
  for ent in doc_i2b2.entities:
    lst_entities.append(ent.text)
  
  lst_entities = list(set(lst_entities))
  str_entities = " | ".join(lst_entities)
  
  return str_entities

In [ ]:
def main():
  lst_modelName = ["T5"]
  lst_model_type = ["w_named_entities"]
  #data_name = "heart-failure"
  data_name = "NLMCXR_reports_ecgen_radiology"
  model_name_prefix = "chest-x-rays"
  lst_inference_type = ["vanilla"]   # determines what to pass during model inference time (testing time)

  # download and initialize a mimic pipeline with an i2b2 NER model
  stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
  nlp_i2b2 = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      for inference_type in lst_inference_type:

        input_fileName = f"{data_name}_w_named_entites_w_facts.jsonl"
        output_filename = f"{data_name}-summaries-{model_type}.jsonl"

        # model path
        MODEL_DIRECTORY_PATH = f"{model_name_prefix}-FINAL-CHECKPTS/{modelName}" 
        model_path = f'{MODEL_DIRECTORY_PATH}/{modelName}-{model_type}-checkpoints/'   # this is checkpoint path in other words

        model, tokenizer = _loadModel(modelName, model_path)
        model = model.to(device)

        SUMMARY_OUTPUT_PATH = f"{data_name}-FINAL-SUMMARIES/{modelName}_{inference_type}_at_inference"
        os.makedirs(SUMMARY_OUTPUT_PATH, exist_ok=True)
        
        with open(f"{input_fileName}") as f:
          
          for idx, line in enumerate(f):
            if idx % 1000 == 0 and idx != 0:
              print("Iteration: ", idx)
            dict_data = json.loads(line)
            
            if inference_type == "w_named_entities":
              input_text = dict_data['finding_entities'] + " [SEP] " + dict_data['finding']
            elif inference_type == "vanilla":
              input_text = dict_data['finding']  # this is the vanilla input configuration at inference time (i.e., without named entities)
            elif inference_type == "w_named_entities_w_facts":
              input_text = dict_data['finding_facts'] + " [SEP] " + dict_data["finding_entities"] + " [SEP] " + dict_data["finding"]

            summary = _generate_abstractive_summary(input_text, model, tokenizer)

            summary_named_entities = _extract_named_entities(summary, nlp_i2b2)  # named entities for the generated summary

            dict_1 = {"indication" : dict_data['indication'],
                      "finding" : dict_data['finding'],
                      "impression" : dict_data['impression'],
                      "finding_entities" : dict_data['finding_entities'],
                      "impression_entities" : dict_data['impression_entities'],
                      "finding_facts" : dict_data['finding_facts'],
                      "impression_facts" : dict_data['impression_facts'],
                      "abstractive_summary" : summary,
                      "abstractive_summary_entities" : summary_named_entities
                      }
                
            with jsonlines.open(f"{SUMMARY_OUTPUT_PATH}/{output_filename}", "a") as writer:
              writer.write(dict_1)
            writer.close()
            
        f.close()

In [ ]:
if __name__ == "__main__":
  main()

2022-01-01 17:36:22 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | mimic   |
| pos             | mimic   |
| lemma           | mimic   |
| depparse        | mimic   |
| ner             | i2b2    |
| backward_charlm | mimic   |
| pretrain        | mimic   |
| forward_charlm  | mimic   |



2022-01-01 17:36:32 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-01-01 17:36:32 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

2022-01-01 17:36:32 INFO: Use device: gpu
2022-01-01 17:36:32 INFO: Loading: tokenize
2022-01-01 17:36:35 INFO: Loading: pos
2022-01-01 17:36:35 INFO: Loading: lemma
2022-01-01 17:36:35 INFO: Loading: depparse
2022-01-01 17:36:36 INFO: Loading: ner
2022-01-01 17:36:36 INFO: Done loading processors!


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Iteration:  1000
Iteration:  2000
Iteration:  3000
